#### 1. Import pandas library

#### 2. Import users table:

#### 3. Rename Id column to userId

#### 4. Import posts table:

#### 5. Rename Id column to postId and OwnerUserId to userId

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

#### 8. How many missing values do you have in your merged dataframe? On which columns?

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 